In [1]:
import argparse
import matplotlib as plt
import pandas as pd
from urllib.request import urlopen
import urllib
from bs4 import BeautifulSoup
import bs4
import requests

THIS CLASS REPRESENTS THE JOB CARDS FROM INDEED.COM SEARCH RESULTS

In [2]:
class JobPost:
    def __init__(self, job_id, job_title, job_company, job_salary, job_date_posted, job_company_link):
        self.job_id = job_id
        self.job_title = job_title
        self.job_company = job_company
        self.job_date_posted = job_date_posted
        self.job_company_link = job_company_link
        self.job_salary = job_salary
    def toString(self):
        return f'{self.job_company_link}\n{self.job_id}\n{self.job_title}\n{self.job_company}\n{self.job_salary}\nposted: {self.job_date_posted}'

PARSE ARGUMENTS FROM CLI

In [3]:
parser = argparse.ArgumentParser()
parser.add_argument("-d", help="Pull latest data on indeed server")
parser.add_argument("-job", help="Specify a job")
parser.add_argument("-n", help="The number")
args = parser.parse_args()

usage: ipykernel_launcher.py [-h] [-d D] [-job JOB] [-n N]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9019 --control=9017 --hb=9016 --Session.signature_scheme="hmac-sha256" --Session.key=b"add3baaf-99bb-4a73-87c3-330a9b2041b6" --shell=9018 --transport="tcp" --iopub=9020 --f=c:\Users\John\AppData\Roaming\jupyter\runtime\kernel-14008hwgrQjYoWv2l.json


SystemExit: 2

c:\Users\John\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3405: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
if args.job is not None:
    job = "none specified"
    job_query = args.job or "data-analyst"

CREATE PAGE REQUESTS WITH CUSTOM FIELDS<br>
RL = "https://www.indeed.com/jobs?q=" + job_query

In [ ]:
URL2 = "https://www.indeed.com/jobs?q=" + job_query + "%20no%20degree%20%2460%2C000&sc=0kf%3Aattr(FCGTU%7CQJZM9%7CUTPWG%252COR)%3B&vjk=e6db7da96543b7b6"

LOAD THE WEBPAGE OR READ FROM HTML FILE

In [ ]:
if(args.d == "pull"):
    html = urllib.request.urlopen(URL2).read()
else:
    html = open("pass1.html", "r", encoding="utf8")
soup = BeautifulSoup(html, "html.parser")

SAVE ANOTHER PASS<br>
with open("pass1.html", "wb") as f:<br>
 f.write(soup.encode(formatter="html"))

In [ ]:
job_lis = soup.find("ul", "jobsearch-ResultsList")
#print(f'found {len(job_lis)} in document')

SEPARATE VALUES RETURNED INTO DIVS THAT CONTAIN BOTH<br>
.jobCard_mainContent AND .jobCardShelfContainer

In [ ]:
result_card_list = soup.find_all("result")

rint(f'found {len(result_card_list)} tags')

FIND THE PARENT TAGS BY CSS CLASS

In [ ]:
results_list = soup.find_all("div", class_="cardOutline")
#print(f"Found {len(results_list)} job search results items \n")

ITERATE THROUGH THE <table> TAGS THAT CONTAIN JOB DATA

In [ ]:
for result in results_list:
    # EXTRACT THE JOB TITLE
    title_h2 = result.find_all("h2", class_="jobTitle")
    a_tag = title_h2[0].find("a")
    title = a_tag.string

    # EXTRACT JOB ID FROM DATA ATTRIBUTE
    job_id = a_tag["data-jk"]

    # EXTRACT THE COMPANY
    company_span = result.find_all("span", class_="companyName")
    for span in company_span:
        if span.string[0] != '+':
            company = span.string
        else:
            company = span[1].string
    
    # EXTRACT THE URL FOR THE JOB APPLICATION
    company_link = result.find("a", class_="jcs-JobTitle")['href']
    company_link = 'http://indeed.com' + company_link
    
    # EXTRACT THE LOCATION
    location_div = result.find("div", "companyLocation")
    location = location_div.string

    # EXTRACT THE SALARY IF IT IS SHOWN ON THE JOBCARD
    salary_divs = result.find("div", "salary-snippet-container")
    salary = "Compensation Not Listed"
    if salary_divs is not None:
        salary_sub_div = salary_divs.find("div")
        salary = "???"
        for s in salary_sub_div.strings:
            if s[0] == '$':
                salary = s
            

    # EXTRACT THE TIME THE JOB WAS POSTED date_posted =
    date_posted = ""
    date_spans = result.find_all("span", class_="date")
    for d_span in date_spans:
        if (d_span.string is not None):
            date_posted = d_span.string

    #print(f'{len(date_span)} spans containing dates were found')
    job = JobPost(job_id, title, company, salary, date_posted, company_link)
    print(job.toString() + "\n")